In [ ]:
import pandas as pd
import pandas_datareader.data as pdr
from datetime import datetime
from datetime import date
from datetime import timedelta
import math

In [ ]:
start = datetime(1900, 1, 1)
end = datetime.now()
sp500 = pdr.get_data_yahoo('^GSPC',start,end,interval='d')
sp500

In [ ]:
sp500.columns

In [ ]:
sp500.index.dtype

In [ ]:
sp500_calculations = sp500[['Adj Close']]
sp500_calculations = sp500_calculations.rename(columns={'Adj Close': 'value'})
sp500_calculations['pct_change'] = sp500_calculations['value'].pct_change()
sp500_calculations

In [ ]:
sp500_calculations = sp500_calculations.copy()
sp500_calculations["cum_return"] = (1 + sp500_calculations["pct_change"]).cumprod()
sp500_calculations

In [ ]:
total_ret = sp500_calculations['cum_return'][-1]
start = datetime.utcfromtimestamp(sp500_calculations.index.values[0].astype('O')/1e9)
end = datetime.utcfromtimestamp(sp500_calculations.index.values[-1].astype('O')/1e9)
period_years = (end - start).days / 365.25
period_years

In [ ]:
CAGR = (total_ret)**(1/period_years) - 1
CAGR

In [ ]:
actions = pdr.DataReader('^GSPC', 'yahoo-actions', start, end)
actions

In [ ]:
# multpl.com
sp500_dividend = pd.read_csv('data/multpl/s-p-500-dividend-yield.csv')
sp500_dividend[:13]

In [ ]:
sp500_dividend.dtypes

In [ ]:
sp500_dividend['Date'] = sp500_dividend['Date'].astype('<M8[ns]')
sp500_dividend.dtypes

In [ ]:
sp500_dividend['Value'] = sp500_dividend['Value'].astype('float')
sp500_dividend.dtypes

In [ ]:
sp500_dividend['Value'] = sp500_dividend['Value'].str.replace('\n\nestimate', '').str.replace('%', '')
sp500_dividend

In [ ]:
sp500_dividend['Value'] = sp500_dividend['Value'].astype('float')
sp500_dividend['Value'] = sp500_dividend['Value'] / 100
sp500_dividend.dtypes

In [ ]:
sp500_dividend

In [ ]:
sp500_dividend = sp500_dividend.set_index('Date').rename(columns={'Value': 'dividend_yield'})
sp500_dividend

In [ ]:
sp500_dividend['dividend_yield'] = sp500_dividend['dividend_yield']/12
sp500_dividend

In [ ]:
sp500_monthly = sp500_calculations.resample('M').last()
sp500_monthly[-13:]

In [ ]:
sp500_monthly['pct_change'] = sp500_monthly['value'].pct_change()
sp500_monthly

In [ ]:
sp500_monthly = sp500_monthly.merge(sp500_dividend, left_index=True, right_index=True)
sp500_monthly

In [ ]:
sp500_monthly = sp500_monthly.drop(columns='cum_return')
sp500_monthly

In [ ]:
sp500_monthly['return'] = sp500_monthly['pct_change'] + sp500_monthly['dividend_yield']
sp500_monthly

In [ ]:
def cagr(_df):
    df = _df.copy()
    df = df.sort_index(ascending=True)
    df["cum_return"] = (1 + df["return"]).cumprod()
    total_ret = df['cum_return'][-1]
    print(df)
    start = datetime.utcfromtimestamp(df.index.values[0].astype('O')/1e9)
    end = datetime.utcfromtimestamp(df.index.values[-1].astype('O')/1e9)
    period_years = (end - start).days / 365.25
    CAGR = (total_ret)**(1/period_years) - 1
    return CAGR

In [ ]:
cagr(sp500_monthly)

In [ ]:
start = datetime.utcfromtimestamp(sp500_monthly.index.values[0].astype('O')/1e9)
end = datetime.utcfromtimestamp(sp500_monthly.index.values[-1].astype('O')/1e9)
period_years = (end - start).days / 365.25
period_years

In [ ]:
sp500_monthly['return'].count()

In [ ]:
ratio_to_annual = sp500_monthly['return'].count() / period_years
ratio_to_annual

In [ ]:
sp500_monthly["return"].std()

In [ ]:
vol = sp500_monthly["return"].std() * math.sqrt(ratio_to_annual)
vol

In [ ]:
def volatility(_df):
    start = datetime.utcfromtimestamp(_df.index.values[0].astype('O')/1e9)
    end = datetime.utcfromtimestamp(_df.index.values[-1].astype('O')/1e9)
    period_years = (end - start).days / 365.25
    ratio_to_annual = _df['return'].count() / period_years
    vol = _df["return"].std() * math.sqrt(ratio_to_annual)
    return vol

volatility(sp500_monthly)

In [ ]:
def sharpe(_df, risk_free_rate):
    ret = cagr(_df)
    vol = volatility(_df)
    sharpe = (ret - risk_free_rate) / vol
    return sharpe

sharpe(sp500_monthly, 0.02)

In [ ]:
sp500_monthly

In [ ]:
df = sp500_monthly.copy()
df["cum_return"] = (1 + df["return"]).cumprod()
df["cum_return_max"] = df["cum_return"].cummax()
df[:20]

In [ ]:
df["drawdown"] = df["cum_return_max"] - df["cum_return"]
df["drawdown_pct"] = df["drawdown"] / df["cum_return_max"]
max_dd = df["drawdown_pct"].max()
max_dd

In [ ]:
df[45:60]

In [ ]:
def max_drawdown(_df):
    df = _df.copy()
    df = df.sort_index(ascending=True)
    df["cum_return"] = (1 + df["return"]).cumprod()
    df["cum_return_max"] = df["cum_return"].cummax()
    df["drawdown"] = df["cum_return_max"] - df["cum_return"]
    df["drawdown_pct"] = df["drawdown"] / df["cum_return_max"]
    max_dd = df.loc[df["drawdown_pct"].idxmax()]
    return {
        'drawdown': max_dd['drawdown_pct'],
        'date': max_dd.name
    }

max_drawdown(sp500_monthly)